In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms

from data.EuroNotes import EuroNotes

from models.BasicCNN import BasicCNN

import numpy as np

import time

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [ ]:
means = np.array([ 0.14588552,  0.26887908,  0.14538361])
stds = np.array([ 0.20122388,  0.2800698 ,  0.20029236])

transformations = transforms.Compose([transforms.ToPILImage(), transforms.RandomCrop([256, 256]), transforms.ToTensor(), transforms.Normalize(means, stds)])

train_set = EuroNotes('../data-augmentation/banknotes_augmented/train', transform=transformations)

train_loader = DataLoader(train_set, batch_size=1, shuffle=True, num_workers=1)

In [ ]:
for i_batch, sample_batched in enumerate(train_loader):
    
    images = sample_batched['image']
    
    img1 = images[0,0,:,:].numpy()
    img2 = images[0,1,:,:].numpy()
    img3 = images[0,2,:,:].numpy()
    img4 = img1 - img2
    
    break

print(img1.shape)
plt.imshow(img1)
plt.show()
plt.imshow(img2)
plt.show()
plt.imshow(img3)
plt.show()
plt.imshow(img4)
plt.show()


